<a href="https://colab.research.google.com/github/czhang2718/LLGM/blob/main/notebooks/base_deepseek_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install transformers
!pip3 install torch
!pip3 install huggingface_hub
!pip3 install flash_attn
!pip3 install accelerate
!pip3 install datasets
!pip3 install evaluate
!pip3 install numpy

In [2]:
model_name = "deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct"
cache_name = ".depsek"
dataset_name = "AnkitAI/CategorizedTextReviews"

In [3]:
from transformers import AutoModel, AutoConfig, AutoTokenizer, TrainingArguments, Trainer
from huggingface_hub import snapshot_download, notebook_login
import torch
from datasets import load_dataset
import numpy as np
import evaluate

In [4]:
notebook_login()

In [ ]:
snapshot_dir = snapshot_download(repo_id=model_name, cache_dir=cache_name)
print(snapshot_dir)

In [ ]:
model_base = AutoModel.from_pretrained(
    model_name,
    cache_dir=cache_name,
    trust_remote_code=True,
    local_files_only=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16
)

In [ ]:
import sys
sys.path.append(".depsek/models--deepseek-ai--DeepSeek-Coder-V2-Lite-Instruct/snapshots/")
!touch {snapshot_dir}/__init__.py
!ls {snapshot_dir}

In [ ]:
from e434a23f91ba5b4923cf6c9d9a238eb4a08e3a11 import modeling_deepseek

In [ ]:
config = AutoConfig.from_pretrained(
    model_name,
    cache_dir=cache_name,
    trust_remote_code=True,
    local_files_only=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16
)
config.hidden_size = 100

In [ ]:
model_classify = modeling_deepseek.DeepseekV2ForSequenceClassification(config)
model_classify.model = model_base

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=cache_name,
    trust_remote_code=True,
    local_files_only=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16
)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["review"], padding="max_length", truncation=True)

In [ ]:
dataset = load_dataset(dataset_name)

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))

In [ ]:
metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

In [ ]:
trainer = Trainer(
    model=model_classify,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()